In [30]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import os
import openpyxl

# reddit_links_to_be_tracked = ['https://old.reddit.com/user/IntelTechnology/?count=26&before=t1_it7xckl','https://old.reddit.com/user/lexhoyos42/?count=26&before=t3_kvebdd']
reddit_links_to_be_tracked = ['https://old.reddit.com/user/lexhoyos42','https://old.reddit.com/user/inteltechnology']
def UniquePosts(next_page,head):

    unique_plink = set()
    
    Next_Page = next_page 
    while True:
        prev_page = Next_Page
        
        resp = requests.get(Next_Page, headers=head)
        print(Next_Page,resp) 
        
        inner_soup = BeautifulSoup(resp.text, 'html.parser')

        links = inner_soup.find_all('a')

        post_link = []
        for link in links:
            post_link.append(link.get('href'))

        

        for i in post_link:
            if i is None:
                continue
            elif (i.startswith('https://') and  'r/intel/comments/' in i) and len(i.split('/')) == 9:
                unique_plink.add(i[:-1] + '.json')


        for i in post_link:
            if i is None:
                continue

#             this line checks if there are exactly two parameters in the URL query string by splitting the query string at the ? character, then splitting the resulting string again at the & character to get a list of key-value pairs.
            if i.startswith('https://old.reddit.com/user/') and 'count=' in i and 'after=' in i and len(i.split('?')[1].split('&')) == 2:
                Next_Page = i
       
        if Next_Page == '' or Next_Page == prev_page:
            break
                    
    return unique_plink

def get_post_data(links_path, output_path,headers):
    with open(links_path, 'r', encoding='utf8') as f:
        links = f.readlines()
        
    file_exists = os.path.exists(output_path)
    
    for i, link in enumerate(links):
        url = link.strip()
        print(f'Processing link {i}: {url}')

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
#             data_dict = data[0]
            reddit_data = {
                'date': data[0]['data']['children'][0]['data']['created_utc'],
                'title': data[0]['data']['children'][0]['data']['title'],
                'description': data[0]['data']['children'][0]['data']['selftext'],
                'author': data[0]['data']['children'][0]['data']['author'],
                'comments': data[0]['data']['children'][0]['data']['num_comments'],
                'score': data[0]['data']['children'][0]['data']['score'],
                'url': data[0]['data']['children'][0]['data']['url'],
            }

    #             print(reddit_data)


        except Exception as e:
            print(f'Error processing link {i}: {e}')
            
        df = pd.DataFrame(reddit_data, index=[i])
        mode = 'w' if i == 0 and not file_exists else 'a'
        
        # write row to csv file
        df.to_csv(output_path, mode=mode, header=mode == 'w', index=False)
        
def main():

    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0'
    }
    
    url_list = reddit_links_to_be_tracked
    
    for url in url_list:
        r = requests.get(url,headers = HEADERS) 
        soup_outer = BeautifulSoup(r.text, 'html.parser') 

        links = soup_outer.find_all('a')

        post_link = []
        for link in links:
            post_link.append(link.get('href')) 

        next_page = url

        try:
            all_links = UniquePosts(next_page,HEADERS)

            with open('links.txt','w') as f:
                for link in all_links:
                    f.write(link +'\n') 
        except Exception as e:
            print(e)

        # get current working directory
        cwd = os.getcwd()

        # join the file name "links.txt" to the current directory path
        links_path = os.path.join(cwd, 'links.txt')

        # set the output path for the csv file
        output_path = os.path.join(cwd, 'reddit_data.csv')

        # call the function with the file paths
        get_post_data(links_path, output_path,HEADERS)

main() 




https://old.reddit.com/user/lexhoyos42 <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=25&after=t1_iwr4dqj <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=50&after=t1_hayvhjv <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=75&after=t1_gxqulcg <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=100&after=t1_gsxfc1v <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=125&after=t1_gstafd2 <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=150&after=t1_gqikny3 <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=175&after=t1_gcrioym <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=200&after=t1_gcqyd5h <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=225&after=t1_gcqntjt <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=250&after=t1_gcqfdtf <Response [200]>
https://old.reddit.com/user/lexhoyos42?count=275&after=t1_gbrpwka <Response [200]>
https://old.reddit.com/user/lexhoy

Processing link 47: https://old.reddit.com/r/intel/comments/k64gib/world_of_warcraft_shadowlands_laptop_graphics.json
Processing link 48: https://old.reddit.com/r/intel/comments/ubfo4t/deleted_by_user.json
Processing link 49: https://old.reddit.com/r/intel/comments/11qt0hu/intel_discord_office_hours_special_guest_liquid.json
Processing link 50: https://old.reddit.com/r/intel/comments/11k6vuk/guys_what_is_this_is_any_of_this_true_please_help.json
Processing link 51: https://old.reddit.com/r/intel/comments/na00cv/ama_with_intel_11th_gen_intel_core_hseries_mobile.json
Processing link 52: https://old.reddit.com/r/intel/comments/memyfv/ama_march_30th_830am_to_300_pm_pst_intel_11th.json
Processing link 53: https://old.reddit.com/r/intel/comments/lhcbrr/made_you_guys_an_evil_corporationesque_intel_logo.json
Processing link 54: https://old.reddit.com/r/intel/comments/lvgqig/intel_discontinues_tuning_protection_plan_for.json
Processing link 55: https://old.reddit.com/r/intel/comments/j6styw/amd

In [ ]:
# Filtering out the data

# load data from csv
df = pd.read_csv('reddit_data.csv')

# re-order columns
df = df[['date', 'url', 'title', 'description', 'author', 'comments','score']]

# convert the timestamp to datetime format
df['date'] = pd.to_datetime(df['date'], unit='s')

# filter to show rows from 2022 and later
df = df[(df['date'].dt.year >= 2022) & (df['author'].isin(['LexHoyos42', 'IntelTechnology']))]

# add calculated column for engagement
df['Engagement(comments + upvotes)'] = df['comments'] + df['score']

# rename columns
df = df.rename(columns={'date':'Date','url': 'URL','title':'Title', 'description': 'Description', 'author': 'Author','comments':'Comments','score':'Upvotes'})

# display final dataframe
print(df)

# write final dataframe to Excel file
df.to_excel('reddit_data.xlsx', index=None) 

In [ ]:
# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
#     'Accept-Language': 'en-US, en;q=0.5',
#     'Accept-Encoding': 'gzip, deflate, br',
#     'Connection': 'keep-alive',
#     'Cache-Control': 'max-age=0'
# }

# # get current working directory
# cwd = os.getcwd()

# # join the file name "links.txt" to the current directory path
# links_path = os.path.join(cwd, 'links.txt')

# # set the output path for the csv file
# output_path = os.path.join(cwd, 'reddit_data.csv')

# def get_post_data(links_path, output_path,headers):
#     with open(links_path, 'r', encoding='utf8') as f:
#         links = f.readlines()

#     for i, link in enumerate(links):
#         url = link.strip()
#         print(f'Processing link {i}: {url}')

#         try:
#             response = requests.get(url, headers=headers)
#             response.raise_for_status()
#             data = response.json()
# #             data_dict = data[0]
#             reddit_data = {
#                 'date': data[0]['data']['children'][0]['data']['created_utc'],
#                 'title': data[0]['data']['children'][0]['data']['title'],
#                 'description': data[0]['data']['children'][0]['data']['selftext'],
#                 'author': data[0]['data']['children'][0]['data']['author'],
#                 'comments': data[0]['data']['children'][0]['data']['num_comments'],
#                 'score': data[0]['data']['children'][0]['data']['score'],
#                 'url': data[0]['data']['children'][0]['data']['url'],
#             }
# #             print(reddit_data)

#             df = pd.DataFrame(reddit_data, index=[i])
#             df.to_csv(output_path, mode='a', header=(i==0))
#         except Exception as e:
#             print(f'Error processing link {i}: {e}')
            
# get_post_data(links_path,output_path,HEADERS)  